# GEFS Wave Global Collection

Reads the Global GEFS Wave grib collection from NODD's S3 bucket and creates a single dataset using fsspec's ReferenceFileSystem

This notebook demonstrates how to generate the reference JSON files using Kerchunk and gribberish. It was adapted from [this notebook](https://nbviewer.org/gist/peterm790/92eb1df3d58ba41d3411f8a840be2452)

In [1]:
import fsspec

Create the virtual filesystem from which to read the NODD s3 bucket. We are going to write the resulting dataset out to our local filesystem, so we leave the write system blank

In [2]:
fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True, use_ssl=False) # For now SSL false is solving my cert issues **shrug**
fs_write = fsspec.filesystem('')

Now, grab all of the files for the given model run. There are 30 ensemble members, so expect 3255 files

In [22]:
gefs_ens_member_files = fs_read.glob('s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p*.global.0p25.*.grib2')

files = sorted(['s3://'+f for f in gefs_ens_member_files if not '.prob.' in f])
print(f'Read {len(files)} GEFS files')

Read 3150 GEFS files


For each file, we will scan it with gribberish and extract its data to a dictionary formatted to the zarr spec. Then we will write that dictionary to a JSON file with the name formatting specified in the `make_json_name` function

In [5]:
import ujson
from gribberish.kerchunk import scan_gribberish

so = {"anon": True, "use_ssl": False}
json_dir = 'gefs_wave/'

def make_json_name(file_url, message_number): #create a unique name for each reference file
    date = file_url.split('/')[3].split('.')[1]
    name = file_url.split('/')[7].split('.')[0:7]
    return f'{json_dir}{date}_{name[0]}_{name[1]}_{name[2]}_{name[4]}_{name[3]}_{name[6]}_message{message_number}.json'

def gen_json(file_url):
    out = scan_gribberish(file_url, storage_options=so, only_vars=['htsgw'], skip=1) 
    for i, message in enumerate(out):
        out_file_name = make_json_name(file_url, i)  # get name
        with fs_write.open(out_file_name, "w") as f: 
            f.write(ujson.dumps(message)) # write to file

In [23]:
import dask
from dask.distributed import Client, progress

client = Client()
futures = client.map(gen_json, files, retries=1)
progress(futures)


/Users/matthewiannucci/Developer/gribberish/python/examples/env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 65039 instead
  warnings.warn(


VBox()

In [24]:
client.shutdown()

Once generated, it can be read back in with xarray or zarr

In [25]:
import xarray as xr

# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo='./gefs_wave/20230706_gefs_wave_t12z_global_p01_f000_message0.json', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False))
ds         

<xarray.Dataset>
Dimensions:    (time: 1, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[s] 2023-07-06T12:00:00
Data variables:
    htsgw      (time, latitude, longitude) float64 ...
Attributes:
    meta:     Generated with gribberishpy

In [ ]:
ds.htsgw.plot()

We can also create a single dataset from the collection of the kerchunked files 

In [26]:
from kerchunk.combine import MultiZarrToZarr
import numpy as np

reference_jsons = fs_write.ls(json_dir) #get list of file names

def fn_to_ens(_index, _fs, _var, fn):
    import re
    groups = re.search(r"\_p(\d{2})", fn).groups()
    return int(groups[0])

# combine individual references into single consolidated reference
mzz = MultiZarrToZarr(reference_jsons,
                        coo_map={'ens': fn_to_ens},
                        coo_dtypes={'ens': np.dtype('int32')},
                        concat_dims = ['ens', 'time'],
                        identical_dims=['latitude', 'longitude'])
d = mzz.translate()

with open('./gefs_wave/gefswave_kerchunk.json', 'w') as f:
    f.write(ujson.dumps(d))

In [27]:
# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo='./gefs_wave/gefswave_kerchunk.json', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={'time':1})

ds                      

<xarray.Dataset>
Dimensions:    (ens: 30, time: 105, latitude: 721, longitude: 1440)
Coordinates:
  * ens        (ens) int32 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[s] 2023-07-06T12:00:00 ... 2023-07-22T12:00:00
Data variables:
    htsgw      (ens, time, latitude, longitude) float64 dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
Attributes:
    meta:     Generated with gribberishpy